In [1]:
import fmrai

# Load dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset('bookcorpus', streaming=True)

In [5]:
next(iter(dataset['train']))

{'text': 'usually , he would be tearing around the living room , playing with his toys .'}

# Create model

In [8]:
from transformers import AutoModel, AutoTokenizer
model = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [100]:
import importlib

import fmrai.instrument
importlib.reload(fmrai.instrument)
from fmrai.instrument import instrument_model

from fmrai.fmrai import Fmrai
import inspect

with fmrai.fmrai() as f:
    f: Fmrai
    
    m = instrument_model(model)
    # print(model.encoder.layer[0].intermediate.intermediate_act_fn)
    
    with f.track() as tracker:
        result = m(**tokenizer('hello', return_tensors='pt'))
        g = tracker.build_graph(result.pooler_output)
        
    print(g)

AssertionError: 